# E7 Project

## 1) 데이터 준비와 전처리

In [17]:
import os
import pandas as pd
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
using_cols = ['user_id', 'movie_id', 'rating']
ratings = ratings[using_cols]
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [18]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [19]:
ratings.rename(columns={'rating':'counts'}, inplace=True)

# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [20]:
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


## 2) 데이터 분석

In [21]:
ratings['user_id'].nunique()

6039

In [22]:
ratings['movie_id'].nunique()

3628

In [23]:
merged_df = pd.merge(ratings, movies, on='movie_id')
merged_df.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,one flew over the cuckoo's nest (1975),Drama


In [24]:

movie_count = merged_df.groupby('title')['title'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [26]:
my_favorite = ['avengers, the (1998)' , 'lord of the rings, the (1978)' ,'saving private ryan (1998)' ,'shawshank redemption, the (1994)' ,'jurassic park (1993)']
new_id = merged_df['user_id'].max() + 1 

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_movielist = pd.DataFrame({'user_id': [new_id]*5, 'title': my_favorite, 'counts':[5]*5})

if not merged_df.isin({'user_id':[new_id]})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    merged_df = merged_df.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

merged_df.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,counts,title,genre
836473,5851,3607.0,5,one little indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,slaughterhouse (1987),Horror
836475,5854,690.0,3,"promise, the (versprechen, das) (1994)",Romance
836476,5938,2909.0,4,"five wives, three secretaries and me (1998)",Documentary
836477,5948,1360.0,5,identification of a woman (identificazione di ...,Drama
0,6041,NaN,5,"avengers, the (1998)",NaN
1,6041,NaN,5,"lord of the rings, the (1978)",NaN
2,6041,NaN,5,saving private ryan (1998),NaN
3,6041,NaN,5,"shawshank redemption, the (1994)",NaN
4,6041,NaN,5,jurassic park (1993),NaN


In [28]:
df = pd.merge(merged_df, movies, on=['title'])
df.tail()

,user_id,movie_id_x,counts,title,genre_x,movie_id_y,genre_y
836478,5851,3607.0,5,one little indian (1973),Comedy|Drama|Western,3607,Comedy|Drama|Western
836479,5854,3026.0,4,slaughterhouse (1987),Horror,3026,Horror
836480,5854,690.0,3,"promise, the (versprechen, das) (1994)",Romance,690,Romance
836481,5938,2909.0,4,"five wives, three secretaries and me (1998)",Documentary,2909,Documentary
836482,5948,1360.0,5,identification of a woman (identificazione di ...,Drama,1360,Drama


In [29]:
df.rename(columns={"movie_id_x":"movie_id", 'genre_x' : 'genre'}, inplace = True)

In [30]:
del df['movie_id_y'], df['genre_y']
df.tail(10)

,user_id,movie_id,counts,title,genre
836473,5494,3530.0,4,smoking/no smoking (1993),Comedy
836474,5556,2198.0,3,modulations (1998),Documentary
836475,5949,2198.0,5,modulations (1998),Documentary
836476,5675,2703.0,3,broken vessels (1998),Drama
836477,5717,2258.0,4,master ninja i (1984),Action
836478,5851,3607.0,5,one little indian (1973),Comedy|Drama|Western
836479,5854,3026.0,4,slaughterhouse (1987),Horror
836480,5854,690.0,3,"promise, the (versprechen, das) (1994)",Romance
836481,5938,2909.0,4,"five wives, three secretaries and me (1998)",Documentary
836482,5948,1360.0,5,identification of a woman (identificazione di ...,Drama


In [35]:
user_unique = df['user_id'].unique()
movie_unique = df['title'].unique()

# 유저, 영화 indexing 하는 코드.
user_to_idx = {v : k for k, v in enumerate(user_unique)}
movie_to_idx = {v : k for k, v in enumerate(movie_unique)}


In [36]:
temp_user_data = df['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(df):
    print('user_id column indexing OK!!')
    df['user_id'] = temp_user_data
else:
    print('user_id column indexing fail')
    
temp_movie_data = df['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(df):
    print('movie column indexing ok!!')
    df['movie_id2'] = temp_movie_data
else:
    print('movie column indexing fail')

df.head(10)

user_id column indexing OK!!
movie column indexing ok!!


,user_id,movie_id,counts,title,genre,movie_id2
0,0,1193.0,5,one flew over the cuckoo's nest (1975),Drama,0
1,1,1193.0,5,one flew over the cuckoo's nest (1975),Drama,0
2,2,1193.0,4,one flew over the cuckoo's nest (1975),Drama,0
3,3,1193.0,4,one flew over the cuckoo's nest (1975),Drama,0
4,4,1193.0,5,one flew over the cuckoo's nest (1975),Drama,0
5,5,1193.0,4,one flew over the cuckoo's nest (1975),Drama,0
6,6,1193.0,5,one flew over the cuckoo's nest (1975),Drama,0
7,7,1193.0,5,one flew over the cuckoo's nest (1975),Drama,0
8,8,1193.0,3,one flew over the cuckoo's nest (1975),Drama,0
9,9,1193.0,5,one flew over the cuckoo's nest (1975),Drama,0


In [37]:
df.tail()

,user_id,movie_id,counts,title,genre,movie_id2
836478,1621,3607.0,5,one little indian (1973),Comedy|Drama|Western,3623
836479,3481,3026.0,4,slaughterhouse (1987),Horror,3624
836480,3481,690.0,3,"promise, the (versprechen, das) (1994)",Romance,3625
836481,4159,2909.0,4,"five wives, three secretaries and me (1998)",Documentary,3626
836482,1648,1360.0,5,identification of a woman (identificazione di ...,Drama,3627


## 4

In [41]:
from scipy.sparse import csr_matrix

num_user = df['user_id'].nunique()
num_movie = df['movie_id2'].nunique()

# print(len(df.counts), len(df.user_id), len(df.movie_id), num_user, num_movie)
csr_data = csr_matrix((df.counts, (df.user_id, df.movie_id2)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>